In [1]:
fname='celerite_003'

n_tta = 6

seed = 0

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from tqdm import tqdm_notebook
import itertools
import pickle as pkl

import autograd
import celerite
from celerite import terms
import scipy.optimize as op
from scipy.optimize import minimize


from multiprocessing import Pool

In [3]:
import random as rn
def init_seeds(seed):

    # The below is necessary for starting Numpy generated random numbers
    # in a well-defined initial state.

    np.random.seed(seed)

    # The below is necessary for starting core Python generated random numbers
    # in a well-defined state.

    rn.seed(seed)


init_seeds(seed)

In [4]:
# eda_031_celerite
def get_gp(train, object_id, expand=True):
    passbands = [0, 1, 2, 3, 4, 5]
    n_param = 2
    res = pd.DataFrame()
    res['object_id'] = [object_id]
    for pb in passbands:
        for i in range(n_param):
            res['celerite_%d_%d' % (pb, i)] = np.NaN
    df0 = train[train.object_id == object_id]
    if df0.hostgal_photoz.mean() == 0:
        return res
    offset = 11
    for pb in range(6):
        if True:
            df = df0[(df0.object_id == object_id) & (df0.passband == pb)]
            flux_err_mean = df.flux_err.mean()
            flux_err_std = df.flux_err.std()
            df = df[df.flux_err <= flux_err_mean + 6*flux_err_std]
            mjd_delta_prev = (df.mjd - df.mjd.shift(1)).fillna(100).values.ravel()
            mjd_delta_next = (df.mjd.shift(-1) - df.mjd).fillna(100).values.ravel()
            x_min = df.mjd.min()
            x_max = df.mjd.max()
            yerr_mean = df.flux_err.mean()
            x = df.mjd.values
            y = df.flux.values
            yerr = df.flux_err
            if expand:
                mjd_delta_prev = np.concatenate((100 * np.ones((offset,)),
                                    mjd_delta_prev,
                                    100 * np.ones((offset,)),
                                  ))
                mjd_delta_next = np.concatenate((100 * np.ones((offset,)),
                                    mjd_delta_next,
                                    100 * np.ones((offset,)),
                                  ))
                x = np.concatenate((np.linspace(x_min-250, x_min -200, offset),
                                    x,
                                    np.linspace(x_max+200, x_max+250, offset),
                                  ))
                y = np.concatenate((np.random.randn(offset) * yerr_mean,
                                    y,
                                    np.random.randn(offset) * yerr_mean
                                   ))
                yerr = np.concatenate((yerr_mean * np.ones(offset),
                                        yerr,
                                        yerr_mean * np.ones(offset)
                                      ))
            #ystd = y.std()
            #y /= ystd
            #yerr = yerr / ystd

            # A Matern32 component
            log_sigma = 0
            log_rho = 0
            eps = 0.001
            bounds = dict(log_sigma=(-15, 15), log_rho=(-15, 15))
            kernel = terms.Matern32Term(log_sigma=log_sigma, log_rho=log_rho, eps=eps, bounds=bounds)
            #kernel.freeze_parameter("eps")  # We don't want to fit for "Q" in this term


            gp = celerite.GP(kernel, mean=0)
            gp.compute(x, yerr)  # You always need to call compute once.

            def neg_log_like(params, y, gp):
                gp.set_parameter_vector(params)
                return -gp.log_likelihood(y)

            def grad_neg_log_like(params, y, gp):
                gp.set_parameter_vector(params)
                return -gp.grad_log_likelihood(y)[1]

            initial_params = gp.get_parameter_vector()
            bounds = gp.get_parameter_bounds()

            r = minimize(neg_log_like, initial_params, jac=grad_neg_log_like, 
                         method="L-BFGS-B", bounds=bounds, args=(y, gp))
            for i in range(n_param):
                res['celerite_%d_%d' % (pb, i)] = r.x[i]
        else:
            continue
    return res

In [5]:
def apply_gp(df, meta):
    df = df[['object_id', 'mjd', 'passband', 'flux', 'flux_err']].merge(meta[['object_id', 'hostgal_photoz']],
                                                           how='left', on='object_id')
    agg =  [get_gp(df, object_id) for object_id in tqdm_notebook(df.object_id.unique())]
    return pd.concat(agg, axis=0)

In [6]:
train = pd.read_csv('../input/training_set.csv')
train.head()

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


In [7]:
meta_cols = ['object_id', 'ddf', 'hostgal_photoz', 'target']
meta_train = pd.read_csv('../input/training_set_metadata.csv')[meta_cols]
meta_train.head()

,object_id,ddf,hostgal_photoz,target
0,615,1,0.0000,92
1,713,1,1.6267,88
2,730,1,0.2262,42
3,745,1,0.2813,90
4,1124,1,0.2415,90


In [8]:
get_gp(train.merge(meta_train, how='left', on='object_id'), 4173)

,object_id,celerite_0_0,celerite_0_1,celerite_1_0,celerite_1_1,celerite_2_0,celerite_2_1,celerite_3_0,celerite_3_1,celerite_4_0,celerite_4_1,celerite_5_0,celerite_5_1
0,4173,4.372327,5.003063,4.66952,5.350874,5.380249,5.955909,4.256575,5.256,3.884235,5.017465,3.452796,4.815897


,object_id,celerite_0_0,celerite_0_1,celerite_1_0,celerite_1_1,celerite_2_0,celerite_2_1,celerite_3_0,celerite_3_1,celerite_4_0,celerite_4_1,celerite_5_0,celerite_5_1
0,4173,4.372327,5.003063,4.66952,5.350874,5.380249,5.955909,4.256575,5.256,3.884235,5.017465,3.440875,4.792469


In [9]:
def work_tta(param):
    (i, fname) = param
    print('starting worker', i)
    train = pd.read_csv('../input/training_set.csv')
    meta_train = pd.read_csv('../input/training_set_metadata.csv')[meta_cols]
    df = train.copy()
    if i > 0:
        init_seeds(i)
        df['flux'] += df['flux_err'] * np.random.randn(*df['flux_err'].shape)
    df = apply_gp(df, meta_train)
    with open('../data/tta_%d_%s.pkl' % (i, fname), 'wb') as file:
        pkl.dump(df, file)  
    print('ending worker', i)
    return 'done'

In [10]:
params = [(i, fname) for i in range(11)]

if 1: 
    pool = Pool(processes=11, maxtasksperchild=1)
    ls   = pool.map( work_tta, params, chunksize=1 )
    pool.close()
else:
    ls = [work_tta(param) for param in params]

starting worker 2
starting worker 0
starting worker 5
starting worker 1
starting worker 3
starting worker 7
starting worker 8
starting worker 4
starting worker 6
starting worker 9
starting worker 10



ending worker 7

ending worker 8



ending worker 3


ending worker 5

ending worker 0
ending worker 9
ending worker 4
ending worker 1

ending worker 10

ending worker 2

ending worker 6


starting worker 1
starting worker 0
starting worker 2
starting worker 3
starting worker 7
starting worker 5
starting worker 4
starting worker 6
starting worker 8
starting worker 10
starting worker 9


In [11]:
def work_test(param):
    (i, fname) = param
    print('starting worker', i)
    with open('../input/test_chunk_%d.csv' %i, 'rb') as file:
        test = pkl.load(file)
    meta_test = pd.read_csv('../input/training_set_metadata.csv')[meta_cols]
    df = apply_gp(test, meta_test)
    with open('../data/test_%d_%s.pkl' % (i, fname), 'wb') as file:
        pkl.dump(df, file)  
    print('ending worker', i)
    return 'done'

In [12]:
params = [(i, fname) for i in range(91)]
params.append((100, fname))

if 1: 
    pool = Pool(processes=20, maxtasksperchild=1)
    ls   = pool.map( work_test, params, chunksize=1 )
    pool.close()
else:
    ls = [work_tta(param) for param in params]

starting worker 0
starting worker 2
starting worker 4
starting worker 3
starting worker 6
starting worker 8
starting worker 5
starting worker 1
starting worker 7
starting worker 15
starting worker 11
starting worker 9
starting worker 10
starting worker 14
starting worker 13
starting worker 12
starting worker 18
starting worker 16
starting worker 19
starting worker 17


/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/celerite/celerite.py:119: RuntimeWarning: invalid value encountered in less
  if check_sorted and np.any(np.diff(t) < 0.0):
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/celerite/celerite.py:119: RuntimeWarning: invalid value encountered in less
  if check_sorted and np.any(np.diff(t) < 0.0):
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/celerite/celerite.py:119: RuntimeWarning: invalid value encountered in less
  if check_sorted and np.any(np.diff(t) < 0.0):
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/celerite/celerite.py:119: RuntimeWarning: invalid value encountered in less
  if check_sorted and np.any(np.diff(t) < 0.0):
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/celerite/celerite.py:119: RuntimeWarning: invalid value encountered in less
  if check_sorted and np.any(np.diff(t) < 0.0):



ending worker 1
starting worker 20


/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/celerite/celerite.py:119: RuntimeWarning: invalid value encountered in less
  if check_sorted and np.any(np.diff(t) < 0.0):
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/celerite/celerite.py:119: RuntimeWarning: invalid value encountered in less
  if check_sorted and np.any(np.diff(t) < 0.0):



ending worker 0
starting worker 21


...


ending worker 80

ending worker 81


/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/celerite/celerite.py:119: RuntimeWarning: invalid value encountered in less
  if check_sorted and np.any(np.diff(t) < 0.0):
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/celerite/celerite.py:119: RuntimeWarning: invalid value encountered in less
  if check_sorted and np.any(np.diff(t) < 0.0):



ending worker 90


/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/celerite/celerite.py:119: RuntimeWarning: invalid value encountered in less
  if check_sorted and np.any(np.diff(t) < 0.0):



ending worker 82

ending worker 88

ending worker 89




ending worker 84
ending worker 85
ending worker 86
ending worker 83

ending worker 87
